In [11]:
import itk
import vtk
from vtk.util import numpy_support
import numpy as np

In [12]:
def itk_to_numpy(image_itk):
    return itk.GetArrayFromImage(image_itk)

def numpy_to_vtk_volume(numpy_array):
    """Convertit un volume numpy 3D en vtkImageData"""
    flat_array = numpy_array.flatten(order='F')
    vtk_data_array = numpy_support.numpy_to_vtk(num_array=flat_array, deep=True, array_type=vtk.VTK_FLOAT)
    
    image = vtk.vtkImageData()
    image.SetDimensions(numpy_array.shape[2], numpy_array.shape[1], numpy_array.shape[0])
    image.SetSpacing(1.0, 1.0, 1.0)
    image.SetOrigin(0.0, 0.0, 0.0)
    image.GetPointData().SetScalars(vtk_data_array)
    return image

In [13]:
class InteractiveImageViewer:
    def __init__(self, volume1, volume2):
        self.volume1 = volume1
        self.volume2 = volume2
        self.shape = volume1.shape
        
        # Position actuelle des coupes
        self.axial_slice = self.shape[0] // 2
        self.coronal_slice = self.shape[1] // 2
        self.sagittal_slice = self.shape[2] // 2
        
        # Conversion en VTK
        self.vtk_volume1 = numpy_to_vtk_volume(volume1)
        self.vtk_volume2 = numpy_to_vtk_volume(volume2)
        
        # Initialisation des composants VTK
        self.reslice_cursors = []
        self.image_actors = []
        self.renderers = []
        
        self.setup_gui()
        self.setup_pipeline()
    
    def setup_pipeline(self):
        # Configuration pour les 6 vues (3 orientations x 2 volumes)
        orientations = ['axial', 'coronal', 'sagittal']
        volumes = [self.vtk_volume1, self.vtk_volume2]
        
        for vol_idx, volume in enumerate(volumes):
            for orient_idx, orientation in enumerate(orientations):
                # Reslice cursor pour chaque vue
                reslice = vtk.vtkImageReslice()
                reslice.SetInputData(volume)
                reslice.SetInterpolationModeToLinear()
                
                # Configuration de l'orientation
                if orientation == 'axial':
                    reslice.SetResliceAxesDirectionCosines(1,0,0, 0,1,0, 0,0,1)
                elif orientation == 'coronal':
                    reslice.SetResliceAxesDirectionCosines(1,0,0, 0,0,1, 0,-1,0)
                else:  # sagittal
                    reslice.SetResliceAxesDirectionCosines(0,1,0, 0,0,1, 1,0,0)
                
                # Mapper et actor
                mapper = vtk.vtkImageMapper()
                mapper.SetInputConnection(reslice.GetOutputPort())
                mapper.SetColorLevel(128)
                mapper.SetColorWindow(256)
                
                actor = vtk.vtkActor2D()
                actor.SetMapper(mapper)
                
                # Ajouter à la liste
                self.reslice_cursors.append(reslice)
                self.image_actors.append(actor)
                
                # Ajouter l'actor au renderer correspondant
                renderer_idx = vol_idx * 3 + orient_idx
                self.renderers[renderer_idx].AddActor(actor)
        
        self.update_slices()
    
    def setup_gui(self):
        # Fenêtre principale
        self.render_window = vtk.vtkRenderWindow()
        self.render_window.SetSize(1800, 900)
        self.render_window.SetWindowName("Navigation Interactive - Avant/Après")
        
        # Interactor
        self.interactor = vtk.vtkRenderWindowInteractor()
        self.render_window.SetInteractor(self.interactor)
        
        # Configuration des viewports (2x3 grid)
        viewports = [
            [0.0, 0.5, 0.33, 1.0],   # Volume1 - Axial
            [0.33, 0.5, 0.66, 1.0],  # Volume1 - Coronal  
            [0.66, 0.5, 1.0, 1.0],   # Volume1 - Sagittal
            [0.0, 0.0, 0.33, 0.5],   # Volume2 - Axial
            [0.33, 0.0, 0.66, 0.5],  # Volume2 - Coronal
            [0.66, 0.0, 1.0, 0.5],   # Volume2 - Sagittal
        ]
        
        titles = [
            "Volume 1 - Axial", "Volume 1 - Coronal", "Volume 1 - Sagittal",
            "Volume 2 - Axial", "Volume 2 - Coronal", "Volume 2 - Sagittal"
        ]
        
        # Créer les renderers
        for i, (viewport, title) in enumerate(zip(viewports, titles)):
            renderer = vtk.vtkRenderer()
            renderer.SetViewport(*viewport)
            renderer.SetBackground(0.1, 0.1, 0.1)
            
            # Ajouter un titre
            text_actor = vtk.vtkTextActor()
            text_actor.SetInput(title)
            text_actor.GetTextProperty().SetFontSize(14)
            text_actor.GetTextProperty().SetColor(1, 1, 1)
            text_actor.SetPosition(10, 10)
            renderer.AddActor(text_actor)
            
            self.renderers.append(renderer)
            self.render_window.AddRenderer(renderer)
        
        # Gestionnaire d'événements clavier
        self.interactor.AddObserver("KeyPressEvent", self.on_key_press)
    
    def update_slices(self):
        """Met à jour la position des coupes pour toutes les vues"""
        orientations = ['axial', 'coronal', 'sagittal']
        slices = [self.axial_slice, self.coronal_slice, self.sagittal_slice]
        
        for vol_idx in range(2):  # 2 volumes
            for orient_idx, (orientation, slice_pos) in enumerate(zip(orientations, slices)):
                reslice_idx = vol_idx * 3 + orient_idx
                reslice = self.reslice_cursors[reslice_idx]
                
                # Position de la coupe selon l'orientation
                if orientation == 'axial':
                    reslice.SetResliceAxesOrigin(0, 0, slice_pos)
                elif orientation == 'coronal':
                    reslice.SetResliceAxesOrigin(0, slice_pos, 0)
                else:  # sagittal
                    reslice.SetResliceAxesOrigin(slice_pos, 0, 0)
                
                reslice.Update()
        
        if hasattr(self, 'render_window'):
            self.render_window.Render()
    
    def on_key_press(self, obj, event):
        """Gestionnaire des événements clavier pour la navigation"""
        key = self.interactor.GetKeySym()
        
        if key == 'Up':
            self.axial_slice = min(self.axial_slice + 1, self.shape[0] - 1)
        elif key == 'Down':
            self.axial_slice = max(self.axial_slice - 1, 0)
        elif key == 'Left':
            self.sagittal_slice = max(self.sagittal_slice - 1, 0)
        elif key == 'Right':
            self.sagittal_slice = min(self.sagittal_slice + 1, self.shape[2] - 1)
        elif key == 'Prior':  # Page Up
            self.coronal_slice = min(self.coronal_slice + 1, self.shape[1] - 1)
        elif key == 'Next':   # Page Down
            self.coronal_slice = max(self.coronal_slice - 1, 0)
        
        self.update_slices()
        print(f"Position: Axial={self.axial_slice}/{self.shape[0]-1}, "
              f"Coronal={self.coronal_slice}/{self.shape[1]-1}, "
              f"Sagittal={self.sagittal_slice}/{self.shape[2]-1}")
    
    def show(self):
        """Affiche la vue interactive"""
        print("Contrôles:")
        print("  ↑/↓ : Navigation axiale")
        print("  ←/→ : Navigation sagittale") 
        print("  Page Up/Down : Navigation coronale")
        print("  'q' : Quitter")
        
        self.render_window.Render()
        self.interactor.Start()

def show_interactive_comparison(volume1, volume2):
    """Fonction principale pour afficher la comparaison interactive"""
    viewer = InteractiveImageViewer(volume1, volume2)
    viewer.show()


In [14]:
# Chargement des volumes avec ITK
image1 = itk.imread("./Data/case6_gre1.nrrd", itk.F)
image2 = itk.imread("./Data/case6_gre2.nrrd", itk.F)

# Conversion ITK → NumPy
array1 = itk_to_numpy(image1)
array2 = itk_to_numpy(image2)

In [15]:
# Affichage interactif avant/après avec navigation
show_interactive_comparison(array1, array2)

Contrôles:
  ↑/↓ : Navigation axiale
  ←/→ : Navigation sagittale
  Page Up/Down : Navigation coronale
  'q' : Quitter
Position: Axial=87/175, Coronal=128/255, Sagittal=128/255
Position: Axial=88/175, Coronal=128/255, Sagittal=128/255
Position: Axial=87/175, Coronal=128/255, Sagittal=128/255
Position: Axial=88/175, Coronal=128/255, Sagittal=128/255
Position: Axial=87/175, Coronal=128/255, Sagittal=128/255
Position: Axial=87/175, Coronal=128/255, Sagittal=127/255
Position: Axial=87/175, Coronal=128/255, Sagittal=128/255
Position: Axial=87/175, Coronal=128/255, Sagittal=127/255
Position: Axial=88/175, Coronal=128/255, Sagittal=127/255
Position: Axial=89/175, Coronal=128/255, Sagittal=127/255
Position: Axial=90/175, Coronal=128/255, Sagittal=127/255
Position: Axial=88/175, Coronal=128/255, Sagittal=127/255
Position: Axial=89/175, Coronal=128/255, Sagittal=127/255
Position: Axial=90/175, Coronal=128/255, Sagittal=127/255
Position: Axial=91/175, Coronal=128/255, Sagittal=127/255
Position: A